# Initial setup w.r.t installation and dependencies

## Setting Google Colab path

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# Changing working directory
import os
base_path='/content/drive/My Drive/Use-Cases/Cigna-usecase/llm-usecases/usecase1/'
os.chdir(base_path)

Mounted at /content/drive


## Install dependencies and import libraries

In [2]:
%pip install --upgrade pip

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
from datasets import Dataset, DatasetDict
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig

In [4]:
# Dataset downloaded from - https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
reviews_path = f'dataset/Reviews.csv'


# Downloading data and analysing

In [5]:
pd_reviews = pd.read_csv(reviews_path)

In [6]:
pd_reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
pd_reviews.groupby('ProductId').head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [8]:
mask_productId = pd_reviews['ProductId'].isin(['B001E4KFG0'])

In [9]:
pd_reviews[mask_productId]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...


In [10]:
len(pd_reviews)

568454

In [11]:
len(pd_reviews.groupby('ProductId'))

74258

### Loading model

In [12]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# assigning to GPU
if "cuda" in device:
  original_model = original_model.to(device)

print(f'Device chosen is : {device}')

Device chosen is : cuda:0


In [13]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [ ]:
rand_indx = np.random.randint(0,len(pd_reviews))
print(f'rand indx is : {rand_indx}')

# working great on following index : 409601,

rand indx is : 60727


In [ ]:
# rand_indx = 1
rand_pd_review_data = pd_reviews.iloc[rand_indx]
# rand_pd_review_data

In [ ]:
review_text = rand_pd_review_data['Text']
gt_summary = rand_pd_review_data['Summary']

print(f'Review Text :\n{review_text} ::\n\nSummary (GT) : \n{gt_summary}')

Review Text :
I am very leery of "energy drinks." I have never tried Red Bull or Rockstar, and I really don't plan on it either. The thought of pumping myself with artificial energy (sugar and caffeine) and then crashing hard is just not very appealing to me. So, I was likewise hesitant with 5-hour Energy.<br /><br />After hearing all of the radio ads, though, and constantly seeing these little red bottles on the gas station counters, I had to give it a shot. I kind of coughed it down the first time as the flavor wasn't real great to me, but I figured it was only 2 ounces and I could get it down. I'm sure glad I did, because I was amazed at what it did for me. Using vitamins and minerals (and the same caffeine dose as in a cup of coffee), I was given energy that lasted and did not cause me to crash!<br /><br />I try to keep some of this stuff on hand (berry is my favorite by the way) and I usually take this when I am feeling like I really need a nap but the nap isn't an option. While i

In [ ]:
prompt = f"""

Summarize the following review.

{review_text}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt').to(device)
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=100,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'*100
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE GT SUMMARY:\n{gt_summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

----------------------------------------------------------------------------------------------------
INPUT PROMPT:


Summarize the following review.

I am very leery of "energy drinks." I have never tried Red Bull or Rockstar, and I really don't plan on it either. The thought of pumping myself with artificial energy (sugar and caffeine) and then crashing hard is just not very appealing to me. So, I was likewise hesitant with 5-hour Energy.<br /><br />After hearing all of the radio ads, though, and constantly seeing these little red bottles on the gas station counters, I had to give it a shot. I kind of coughed it down the first time as the flavor wasn't real great to me, but I figured it was only 2 ounces and I could get it down. I'm sure glad I did, because I was amazed at what it did for me. Using vitamins and minerals (and the same caffeine dose as in a cup of coffee), I was given energy that lasted and did not cause me to crash!<br /><br />I try to keep some of this stuff on hand (

### Full fine-tuning for this specific task


In [14]:
# convert pandas dataset to 'datasets'
pd_reviews_subset = pd_reviews.sample(frac=0.01)

print(f'# rows in sample : {len(pd_reviews_subset)}')

datasets = Dataset.from_pandas(pd_reviews_subset)

train_test_dataset = datasets.train_test_split(test_size=0.2)
# Split the 10% test + valid in `half test, half valid`
test_valid = train_test_dataset['test'].train_test_split(test_size=0.5)

# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_test_dataset['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

train_test_valid_dataset

# rows in sample : 5685


DatasetDict({
    train: Dataset({
        features: ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text', '__index_level_0__'],
        num_rows: 4548
    })
    test: Dataset({
        features: ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text', '__index_level_0__'],
        num_rows: 569
    })
    valid: Dataset({
        features: ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text', '__index_level_0__'],
        num_rows: 568
    })
})

In [15]:
def tokenize_function(example):
    max_length = 512
    start_prompt = 'Summarize the customer feedback.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + review + end_prompt for review in example["Text"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt").input_ids.to(device)
    example['labels'] = tokenizer(example["Summary"], padding="max_length", max_length=max_length, truncation=True, return_tensors="pt").input_ids.to(device)

    return example

In [16]:
tokenized_datasets = train_test_valid_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text', '__index_level_0__'])

Map:   0%|          | 0/4548 [00:00<?, ? examples/s]

Map:   0%|          | 0/569 [00:00<?, ? examples/s]

Map:   0%|          | 0/568 [00:00<?, ? examples/s]

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['valid'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (4548, 2)
Validation: (568, 2)
Test: (569, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 4548
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 569
    })
    valid: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 568
    })
})


## Full fine-tuning

In [ ]:
output_dir = f'./checkpoints/cust-review-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    auto_find_batch_size=True,
    logging_steps=500,
    warmup_ratio=0.8,
    eval_steps=500,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=3,
#     logging_steps=500,
#     max_steps=1000,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
    lr_scheduler_type='cosine',
#     metric_for_best_model='map3',
    report_to='none'
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid']
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.049400,0.042114
2,0.044400,0.041691
3,0.056300,0.050491


TrainOutput(global_step=3411, training_loss=1.1445536886062555, metrics={'train_runtime': 5433.1194, 'train_samples_per_second': 2.511, 'train_steps_per_second': 0.628, 'total_flos': 9342829316800512.0, 'train_loss': 1.1445536886062555, 'epoch': 3.0})

In [ ]:
fine_tuned_model_path = f'./checkpoints/fine_tunned_checkpoints/'
# saving models
trainer.model.save_pretrained(fine_tuned_model_path)
tokenizer.save_pretrained(fine_tuned_model_path)

('./checkpoints/fine_tunned_checkpoints/tokenizer_config.json',
 './checkpoints/fine_tunned_checkpoints/special_tokens_map.json',
 './checkpoints/fine_tunned_checkpoints/tokenizer.json')

In [ ]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(fine_tuned_model_path, torch_dtype=torch.bfloat16).to(device)

In [ ]:
prompt = f"""

Summarize the following review.

{review_text}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

dash_line = '--'*50
print(dash_line)
print(f'BASELINE GT SUMMARY:\n{gt_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

----------------------------------------------------------------------------------------------------
BASELINE GT SUMMARY:
Earl Grey of Earl Grey +++
----------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
Earl Grey is the best!
----------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
Earl Grey is the best!


## PEFT fine-tuning

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [ ]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 884736
all model parameters: 248462592
percentage of trainable model parameters: 0.36%


In [ ]:
output_dir = f'./checkpoints/cust-review-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    auto_find_batch_size=True,
    logging_steps=100,
    warmup_ratio=0.8,
    eval_steps=100,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=3,
#     logging_steps=500,
#     max_steps=1000,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
    lr_scheduler_type='cosine',
#     metric_for_best_model='map3',
    report_to='none'
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid'],
    tokenizer=tokenizer,
#     compute_metrics=compute_metrics_l,
)

In [ ]:
peft_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.136500,0.064226
2,0.062600,0.048696
3,0.057200,0.046481


TrainOutput(global_step=1707, training_loss=2.637348481207894, metrics={'train_runtime': 4095.2879, 'train_samples_per_second': 3.332, 'train_steps_per_second': 0.417, 'total_flos': 9379912467087360.0, 'train_loss': 2.637348481207894, 'epoch': 3.0})

In [ ]:
peft_model_path = f'./checkpoints/peft_fine_tunned_checkpoints/'
# saving models
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('./checkpoints/peft_fine_tunned_checkpoints/tokenizer_config.json',
 './checkpoints/peft_fine_tunned_checkpoints/special_tokens_map.json',
 './checkpoints/peft_fine_tunned_checkpoints/tokenizer.json')

In [17]:
peft_model_path = f'./checkpoints/peft_fine_tunned_checkpoints/'

# Now loading with trained peft model
peft_model_base = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       peft_model_path,
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False).to(device)

In [18]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 248462592
percentage of trainable model parameters: 0.00%


## Evaluation of the model trained on

In [ ]:
train_test_valid_dataset['test'][0]['Text']

"This coffee is fantastic, but does anyone else notice that it's cheaper to buy two 24 packs than it is to buy one 48 pack? That's a little confusing..."

In [19]:
reviews = train_test_valid_dataset['test'][0:10]['Text']
gt_baseline_summaries = train_test_valid_dataset['test'][0:10]['Summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, review in enumerate(reviews):
    prompt = f"""
Summarize the following conversation.

{review}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    human_baseline_text_output = gt_baseline_summaries[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    # instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    # instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    # instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(gt_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['gt_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

,gt_baseline_summaries,original_model_summaries,peft_model_summaries
0,Delicious,Love this K-Cup!,Love Love Love this K-Cup. Amazon is the only ...
1,Made in the USA!,Great product!,My dogs love Zuke's Z-Filets!
2,Great popper popcorn!,Great popcorn!,Great popcorn!
3,Will purchase from this seller again,Great seller!,Great seller
4,Love &lt;3,Choose a bar with a chocolate bar.,Try the chocolate bar.
5,Hazelnut coffee--Gloria Jean's,Great coffee.,Great coffee!
6,Too sugary,I bought these Special K bars because of the b...,I'm not a fan of these bars.
7,Be aware! Different product...,Not a good product.,Not a good product
8,Good as are all Torani products,I'm not sure what the product is about.,I'm not sure what the product is about.
9,Great gum,Great cinnamon flavor and I much prefer the in...,Great cinnamon flavor


In [20]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=gt_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=gt_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.18792063492063493, 'rouge2': 0.004651162790697674, 'rougeL': 0.1868968253968254, 'rougeLsum': 0.18492063492063496}
PEFT MODEL:
{'rouge1': 0.21202380952380956, 'rouge2': 0.0, 'rougeL': 0.2119047619047619, 'rougeLsum': 0.20857142857142857}
